# Load Dataset

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
!unzip -q "/content/MyDrive/MyDrive/UMHackathon/UMHackathon 2021 Finance Dataset.zip" -d "dataset" 

In [3]:
import pandas as pd
import glob
import os
import dateutil.parser
import functools
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime
import random

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Cleaning

In [4]:
dataset_dir = "/content/dataset/BPAM Evaluated Prices Rated Range Prices (LT)"

# Merge both files (BPANERP & BPAMERS) together as a single pandas frame
def read_all_csvs(dataset_dir):
  sub_dirs = [f for f in os.listdir(dataset_dir) if not os.path.isfile(f)]
  df_BPAMERP = pd.DataFrame()
  df_BPAMERS = pd.DataFrame()
  df_joined = pd.DataFrame()
  for sub_dir in sub_dirs:
    full_path = os.path.join(dataset_dir, sub_dir)
    for file_name in os.listdir(full_path):
      full_file_name = os.path.join(full_path, file_name)
      if "BPAMERP" in file_name:
        temp_df_BPAMERP =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERP = pd.concat([df_BPAMERP, temp_df_BPAMERP])
      elif "BPAMERS" in file_name:
        temp_df_BPAMERS =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERS = pd.concat([df_BPAMERS, temp_df_BPAMERS])
    df_joined = pd.concat([df_joined, temp_df_BPAMERP.merge(temp_df_BPAMERS, on = ['STOCK CODE','ISIN CODE','STOCK NAME'], how = 'inner')])
  return df_BPAMERP.reset_index(drop=True), df_BPAMERS.reset_index(drop=True), df_joined.sort_values(by=['VALUE DATE']).reset_index(drop=True)

df_BPAMERP, df_BPAMERS, df_joined = read_all_csvs(dataset_dir)
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34206 entries, 0 to 34205
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   STOCK CODE                                34206 non-null  object 
 1   ISIN CODE                                 34206 non-null  object 
 2   STOCK NAME                                34206 non-null  object 
 3   VALUE DATE                                34206 non-null  object 
 4   EVAL UPPER THRESHOLD YIELD                34206 non-null  float64
 5   EVAL MID YIELD                            34206 non-null  float64
 6   EVAL LOWER THRESHOLD YIELD                34206 non-null  float64
 7   EVAL LOWER THRESHOLD PRICE                34206 non-null  float64
 8   EVAL MID PRICE                            34206 non-null  float64
 9   EVAL UPPER THRESHOLD PRICE                34206 non-null  float64
 10  MODIFIED DURATION                 

In [5]:
# parse datetime data type
date_columns = [column for column in df_joined.columns if "DATE" in column]
for date_column in date_columns:
  df_joined[date_column] = pd.to_datetime(df_joined[date_column], infer_datetime_format=True)  

# parse bool data type
d = {'Y': True, 'N': False}
df_joined["CALLABLE/PUTTABLE"] = df_joined["CALLABLE/PUTTABLE"].map(d)
df_joined["CONVERTIBLE/EXCHANGABLE"] = df_joined["CONVERTIBLE/EXCHANGABLE"].map(d)

# parse category data type
obj_columns = df_joined.select_dtypes(include='object').columns
for obj_column in obj_columns:
  df_joined[obj_column] = df_joined[obj_column].astype('category')

# show the memory usage
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34206 entries, 0 to 34205
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   STOCK CODE                                34206 non-null  category      
 1   ISIN CODE                                 34206 non-null  category      
 2   STOCK NAME                                34206 non-null  category      
 3   VALUE DATE                                34206 non-null  datetime64[ns]
 4   EVAL UPPER THRESHOLD YIELD                34206 non-null  float64       
 5   EVAL MID YIELD                            34206 non-null  float64       
 6   EVAL LOWER THRESHOLD YIELD                34206 non-null  float64       
 7   EVAL LOWER THRESHOLD PRICE                34206 non-null  float64       
 8   EVAL MID PRICE                            34206 non-null  float64       
 9   EVAL UPPER THRESHOLD PRICE  

In [6]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
PRINCIPLE                                       0
BOND TYPE                                       0
BOND CLASS                                      0
CONVERTIBLE/EXCHANGABLE                         0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
ISSUER NAME                                     0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
BOND ISSUE AMOUNT(MYR MIL)                      0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
FACILITY CODE                                   0
RATING                                          0


In [7]:
# impute null value
df_joined["COUPON FREQUENCY"].fillna(0, inplace = True)
df_joined["NEXT COUPON RATE"].fillna(0, inplace = True)
df_joined["PREVIOUS COUPON RATE"].fillna(0, inplace = True)

In [8]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
BOND CLASS                                      0
CONVERTIBLE/EXCHANGABLE                         0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
COUPON FREQUENCY                                0
PREVIOUS COUPON RATE                            0
NEXT COUPON RATE                                0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
BOND ISSUE AMOUNT(MYR MIL)                      0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
BOND TYPE                                       0
PRINCIPLE                                       0


# Features Engineering

In [9]:
# extract month data from [VALUE DATE] 
# eg: VALUE DATE 2020-04-01 refers to March, that is why we need to subtract by 1
df_joined["VALUE DATE MONTH"] = df_joined["VALUE DATE"].dt.to_period('M') - 1
df_joined["VALUE DATE MONTH"]

0        2020-03
1        2020-03
2        2020-03
3        2020-03
4        2020-03
          ...   
34201    2020-04
34202    2020-04
34203    2020-04
34204    2020-04
34205    2020-04
Name: VALUE DATE MONTH, Length: 34206, dtype: period[M]

In [10]:
# calculate maturity duration in days 
df_joined["MATURITY DURATION"] = (df_joined["MATURITY DATE"] - df_joined["ISSUE DATE"]).dt.days
df_joined["MATURITY DURATION"]

0         3653
1         9131
2         9129
3         9129
4         9130
         ...  
34201     9130
34202     9129
34203     9129
34204    10956
34205     1277
Name: MATURITY DURATION, Length: 34206, dtype: int64

In [11]:
# calculate accured interest
# assume ACTACT and ACTBOTH are the same
def cal_accured_interest(bond):
  prev_date = bond["PREVIOUS PAYMENT DATE"] if bond["PREVIOUS PAYMENT DATE"] is not pd.NaT else bond["ISSUE DATE"]
  num_of_days_since_last_coupon_payment = float((bond["VALUE DATE"] - prev_date).days)
  if bond["DAY COUNT BASIS"] in ["ACTACT","ACTBOTH"]:
    payment_period = float((bond["NEXT PAYMENT DATE"] - prev_date).days)
    if bond["COUPON FREQUENCY"] == 0:
      return 0
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) / bond["COUPON FREQUENCY"] * num_of_days_since_last_coupon_payment / payment_period
    return accured_interest
  elif bond["DAY COUNT BASIS"] in ["ACT365"]:
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) * num_of_days_since_last_coupon_payment / 365
    return accured_interest

df_joined["ACCRUED INTEREST"] = df_joined.apply(cal_accured_interest, axis=1)
df_joined["ACCRUED INTEREST"]

0        1.660548
1        0.125753
2        1.823562
3        1.930685
4        0.574247
           ...   
34201    1.048000
34202    2.402630
34203    2.293699
34204    2.462466
34205    0.849863
Name: ACCRUED INTEREST, Length: 34206, dtype: float64

In [12]:
# set opr movement
OPR_movement = { 
    "7 Jul 2020": 1.75,
    "5 May 2020": 2,
    "3 Mar 2020": 2.5,    
    "22 Jan 2020": 2.75,
    "7 May 2019": 3
}

def assign_opr(value_date):
  for date, opr in OPR_movement.items(): 
    date = dateutil.parser.parse(date) 
    if value_date >= date:
      return opr

df_joined["OPR movement"] = df_joined["VALUE DATE"].apply(assign_opr)
df_joined["OPR movement"]

0        2.5
1        2.5
2        2.5
3        2.5
4        2.5
        ... 
34201    2.5
34202    2.5
34203    2.5
34204    2.5
34205    2.5
Name: OPR movement, Length: 34206, dtype: float64

In [14]:
# calculate changes In EVAL MID Price & moving average changes In EVAL MID Price for the past 5 months
def transform_eval_mid_price(df_joined):
  temp_df_joined = pd.DataFrame()
  for stock_code in df_joined["STOCK CODE"].unique():
    temp_df = df_joined[df_joined["STOCK CODE"] == stock_code].sort_values("VALUE DATE MONTH")
    temp_df["CHANGES IN EVAL MID PRICE"] = temp_df["EVAL MID PRICE"].diff()
    temp_df["MA-5 CHANGES IN EVAL MID PRICE"] = temp_df["CHANGES IN EVAL MID PRICE"].rolling(5,1).mean()
    temp_df_joined = pd.concat([temp_df_joined, temp_df])
    
  return temp_df_joined

df_joined = transform_eval_mid_price(df_joined)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR movement,CHANGES IN EVAL MID PRICE,MA-5 CHANGES IN EVAL MID PRICE
18435,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2019-12-02,4.171,4.132,4.093,100.070,100.385,100.702,8.047,76.931,-0.002,0.000,0.002,0.016,-0.001,-0.017,2.0,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2019-11,3653,0.274849,3.00,NaN,NaN
19829,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-01-02,4.165,4.130,4.095,100.116,100.397,100.679,7.964,75.561,0.000,0.000,0.000,0.000,0.000,0.000,1.7,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2019-12,3653,0.629863,3.00,0.012,0.01200
28595,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-02-03,3.914,3.868,3.822,102.136,102.512,102.889,7.909,74.609,-0.026,-0.027,-0.028,0.210,0.219,0.228,1.7,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-01,3653,0.996329,2.75,2.115,1.06350
22752,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-03-02,3.791,3.722,3.653,103.124,103.691,104.262,7.852,73.635,-0.028,-0.029,-0.030,0.227,0.236,0.246,2.7,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-02,3653,1.316986,2.75,1.179,1.10200
0,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-04-01,4.543,4.448,4.353,97.196,97.920,98.650,7.684,71.122,-0.023,-0.024,-0.025,0.175,0.184,0.193,3.0,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-03,3653,1.660548,2.50,-5.771,-0.61625
32751,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-05-04,3.943,3.843,3.743,101.864,102.663,103.470,7.668,70.721,-0.040,-0.040,-0.040,0.316,0.318,0.321,3.3,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2020-04-17,False,False,Reaffirm,2020-04,3653,2.038466,2.50,4.743,0.45560
7806,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche